Esto es el tutorial de openai para function calling

https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models

# How to call functions with chat models

Este cuaderno explica cómo utilizar la <span style="color: rgb(0,255,0);">***Chat Completions API***</span> en combinación con funciones externas para ampliar las capacidades de los modelos GPT.

<span style="color: rgb(0,255,0);">***Tools***</span> es un parámetro opcional de la <span style="color: rgb(0,255,0);">***Chat Completions API***</span> que puede utilizarse para proporcionar especificaciones de función. El propósito de esto es permitir que los modelos generen argumentos de función que se adhieran a las especificaciones proporcionadas. Tenga en cuenta que EN ESTE EJEMPLO la API no ejecutará ninguna llamada a funciones. Corresponde a los desarrolladores ejecutar las llamadas a funciones utilizando los outputs del modelo.

Dentro del parámetro <span style="color: rgb(0,255,0);">***tools***</span>, si se proporciona el parámetro <span style="color: rgb(0,255,0);">***functions***</span>, el modelo decidirá por defecto cuándo es apropiado utilizar una de las funciones. La API puede ser forzada a utilizar una función específica estableciendo el parámetro <span style="color: rgb(0,255,0);">***tool_choice***</span> a <span style="color: rgb(0,255,0);">***{"name":"< insert-function-name >"}***</span>. También se puede forzar a la API a no utilizar ninguna función estableciendo el parámetro <span style="color: rgb(0,255,0);">***tool_choice***</span> como <span style="color: rgb(0,255,0);">***"none"***</span>. Si se utiliza una función, la salida contendrá <span style="color: rgb(0,255,0);">***"finish_reason": "function_call"***</span> en la respuesta, así como un objeto <span style="color: rgb(0,255,0);">***tool_choice***</span> que tiene el nombre de la función y los argumentos de función generados.

Este cuaderno contiene las 2 secciones siguientes:

1. **Cómo generar argumentos de función**: Especificar un conjunto de funciones y utilizar la API para generar argumentos de función.

2. **Cómo llamar a funciones con argumentos generados por el modelo**: Cierre el bucle ejecutando realmente funciones con argumentos generados por el modelo.

# Cómo generar argumentos de función

In [1]:
# !pip install scipy
# !pip install tenacity
# !pip install tiktoken
# !pip install termcolor 
# !pip install openai
# !pip install requests

In [1]:
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
import os
from dotenv import load_dotenv

# Cargar variables de entorno desde .env
load_dotenv()

# Acceder a la API key
api_key = os.getenv("API_KEY")
openai.api_key = api_key

## Utilidades

En primer lugar, vamos a definir algunas utilidades para realizar llamadas a la <span style="color: rgb(0,255,0);">***Chat Completions API***</span> y para mantener y realizar un seguimiento del estado de la conversación.

In [2]:
#@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + api,      # openai.api_key
    }

    json_data = {"model": model, "messages": messages}

    if tools is not None:
        json_data.update({"tools": tools})

    if tool_choice is not None:
        json_data.update({"tool_choice": tool_choice})

    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "tool": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "tool":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


## Conceptos básicos

### tools

Vamos a crear algunas especificaciones de función para interactuar con una hipotética API meteorológica. Pasaremos estas especificaciones de función a la <span style="color: rgb(0,255,0);">***Chat Completions API***</span>  para generar argumentos de función que se adhieran a la especificación.

In [3]:
tools = [
    {   
        "type"    : "function",                            # herramienta 1 de tipo funcion
        "function": {

            "name"       : "get_current_weather",          # nombre de la funcion
            "description": "Get the current weather",      # descripción de lo que la funcion
            "parameters" : {                               # parametros que acepta la funcion

                "type"      :"object",                     # indica que los parámtros son un objeto
                "properties": {                            # propiedades (parametros de la función)

                    "location": {                          # parametro 1 location
                        "type"       : "string",           # el tipo de dato
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format"  : {                          # parametro 2 formato

                        "type"       : "string",                    # tipo
                        "enum"       : ["celsius", "fahrenheit"],   # valores permitidos
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required"  : ["location", "format"],      # cuales de los parametros son requeridos u obligatorios
            },
        }
    },
    {
        "type"    : "function",                            # herramienta 2 de tipo función
        "function": {

            "name"       : "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters" : {

                "type"      : "object",
                "properties": {

                    "location": {

                        "type"       : "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },

                    "format"   : {

                        "type"       : "string",
                        "enum"       : ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },

                    "num_days": {

                        "type"       : "integer",
                        "description": "The number of days to forecast",
                    }
                },

                "required": ["location", "format", "num_days"]
            },
        }
    },
]

#### Explicación

- Este código define dos herramientas que pueden ser llamadas como parte de una conversación utilizando la API de OpenAI.
- Cada herramienta tiene un nombre, una descripción, y define sus parámetros.
- Los parámetros son definidos como un objeto con propiedades específicas, incluyendo su tipo, valores permitidos (en caso de que sea necesario), y una descripción.
- También se especifican los parámetros requeridos para cada función.

### Message

If we prompt the model about the current weather, it will respond with some clarifying questions.

In [4]:
# If we prompt the model about the current weather, it will respond with some clarifying questions.
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user",   "content": "What's the weather like today"})
chat_response = chat_completion_request(
    messages, 
    tools=tools
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

{'role': 'assistant',
 'content': 'Sure, could you please let me know your current location?'}

Once we provide the missing information, it will generate the appropriate function arguments for us.

In [5]:
messages.append({"role": "user", "content": "I'm in Glasgow, Scotland."})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message


{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_PHypElMcP7XA4MGwVYdmvrWa',
   'type': 'function',
   'function': {'name': 'get_current_weather',
    'arguments': '{\n  "location": "Glasgow, Scotland",\n  "format": "celsius"\n}'}}]}

By prompting it differently, we can get it to target the other function we've told it about.

In [6]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user",   "content": "what is the weather going to be like in Glasgow, Scotland over the next x days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message


{'role': 'assistant',
 'content': "Sure, I can help you with that. Please provide the value of 'x' to get the weather forecast for the next 'x' days in Glasgow, Scotland."}

Once again, the model is asking us for clarification because it doesn't have enough information yet. In this case it already knows the location for the forecast, but it needs to know how many days are required in the forecast.

In [7]:
messages.append({"role": "user", "content": "5 days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
chat_response.json()["choices"][0]

{'index': 0,
 'message': {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_vbvwnYGdD5Y3wfPEsNbjieAr',
    'type': 'function',
    'function': {'name': 'get_n_day_weather_forecast',
     'arguments': '{\n  "location": "Glasgow, Scotland",\n  "format": "celsius",\n  "num_days": 5\n}'}}]},
 'finish_reason': 'tool_calls'}

### Forzar el uso de funciones específicas o ninguna función

Podemos forzar al modelo a utilizar una función específica, por ejemplo <span style="color: rgb(0,255,0);">***get_n_day_weather_forecast***</span>  utilizando el argumento <span style="color: rgb(0,255,0);">***function_call***</span> . Al hacerlo, forzamos al modelo a hacer suposiciones sobre cómo utilizarla.

In [8]:
# in this cell we force the model to use get_n_day_weather_forecast
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, 
    tools=tools, 
    tool_choice={"type": "function", "function": {"name": "get_n_day_weather_forecast"}}
)
chat_response.json()["choices"][0]["message"]

{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_m063WvD7rGlK01fCYAVk0Wzr',
   'type': 'function',
   'function': {'name': 'get_n_day_weather_forecast',
    'arguments': '{\n  "location": "Toronto, Canada",\n  "format": "celsius",\n  "num_days": 1\n}'}}]}

In [9]:
# if we don't force the model to use get_n_day_weather_forecast it may not
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, 
    tools=tools
)
chat_response.json()["choices"][0]["message"]


{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_Mi03xO2FknakEqNEUtaFb3Pn',
   'type': 'function',
   'function': {'name': 'get_current_weather',
    'arguments': '{\n  "location": "Toronto, Canada",\n  "format": "celsius"\n}'}}]}

También podemos forzar al modelo a no utilizar ninguna función. De este modo evitamos que produzca una llamada a función adecuada.

In [10]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me the current weather (use Celcius) for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, 
    tools=tools, 
    tool_choice="none"
)

chat_response.json()["choices"][0]["message"]

{'role': 'assistant',
 'content': 'Please use the "get_current_weather" function to get the current weather for Toronto, Canada in Celsius.'}

### Llamada de funciones en paralelo - Parallel function calling

Los modelos más recientes como gpt-4-1106-preview o gpt-3.5-turbo-1106 pueden llamar a múltiples funciones en un turno.

In [11]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in San Francisco and Glasgow over the next 4 days"})
chat_response = chat_completion_request(
    
    messages, 
    tools=tools, 
    model='gpt-3.5-turbo-1106'
)

chat_response.json()
assistant_message = chat_response.json()["choices"][0]["message"]['tool_calls']
assistant_message

[{'id': 'call_0XAFYmCPTS2vUMdcJFApWvyU',
  'type': 'function',
  'function': {'name': 'get_n_day_weather_forecast',
   'arguments': '{"location": "San Francisco, CA", "format": "celsius", "num_days": 4}'}},
 {'id': 'call_aoANwTwMHsU0QOzg9QFG5HBE',
  'type': 'function',
  'function': {'name': 'get_n_day_weather_forecast',
   'arguments': '{"location": "Glasgow", "format": "celsius", "num_days": 4}'}}]

## Cómo llamar a funciones con argumentos generados por el modelo

En nuestro siguiente ejemplo, demostraremos cómo ejecutar funciones cuyas entradas son generadas por el modelo, y usaremos esto para implementar un agente que pueda responder preguntas por nosotros sobre una base de datos. Para simplificar utilizaremos la base de datos de ejemplo Chinook.

Nota: La generación de SQL puede ser de alto riesgo en un entorno de producción, ya que los modelos no son perfectamente fiables a la hora de generar SQL correcto.

### Especificación de una función para ejecutar consultas SQL

Primero vamos a definir algunas funciones útiles para extraer datos de una base de datos SQLite.

In [12]:
import sqlite3

conn = sqlite3.connect("data/Chinook.db")
print("Opened database successfully")

OperationalError: unable to open database file

In [15]:
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names


def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names


def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts


Ahora puede utilizar estas funciones de utilidad para extraer una representación del esquema de la base de datos.

In [16]:
database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

NameError: name 'conn' is not defined

Como antes, definiremos una especificación de función para la función para la que queremos que la API genere argumentos. Observa que estamos insertando el esquema de la base de datos en la especificación de la función. Esto será importante para el modelo a conocer.

In [13]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "ask_database",
            "description": "Use this function to answer user questions about music. Input should be a fully formed SQL query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": f"""
                                SQL query extracting info to answer the user's question.
                                SQL should be written using this database schema:
                                {database_schema_string}
                                The query should be returned in plain text, not in JSON.
                                """,
                    }
                },
                "required": ["query"],
            },
        }
    }
]

NameError: name 'database_schema_string' is not defined

### Ejecución de consultas SQL

Ahora vamos a implementar la función que realmente ejecutará las consultas contra la base de datos.

In [ ]:
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

def execute_function_call(message):
    if message["tool_calls"][0]["function"]["name"] == "ask_database":
        query = json.loads(message["tool_calls"][0]["function"]["arguments"])["query"]
        results = ask_database(conn, query)
    else:
        results = f"Error: function {message['tool_calls'][0]['function']['name']} does not exist"
    return results

In [ ]:
messages = []
messages.append({"role": "system", "content": "Answer user questions by generating SQL queries against the Chinook Music Database."})
messages.append({"role": "user", "content": "Hi, who are the top 5 artists by number of tracks?"})
chat_response = chat_completion_request(messages, tools)
assistant_message = chat_response.json()["choices"][0]["message"]
assistant_message['content'] = str(assistant_message["tool_calls"][0]["function"])
messages.append(assistant_message)
if assistant_message.get("tool_calls"):
    results = execute_function_call(assistant_message)
    messages.append({"role": "tool", "tool_call_id": assistant_message["tool_calls"][0]['id'], "name": assistant_message["tool_calls"][0]["function"]["name"], "content": results})
pretty_print_conversation(messages)

In [ ]:
messages.append({"role": "user", "content": "What is the name of the album with the most tracks?"})
chat_response = chat_completion_request(messages, tools)
assistant_message = chat_response.json()["choices"][0]["message"]
assistant_message['content'] = str(assistant_message["tool_calls"][0]["function"])
messages.append(assistant_message)
if assistant_message.get("tool_calls"):
    results = execute_function_call(assistant_message)
    messages.append({"role": "tool", "tool_call_id": assistant_message["tool_calls"][0]['id'], "name": assistant_message["tool_calls"][0]["function"]["name"], "content": results})
pretty_print_conversation(messages)